# Classification of JIRA comments to find commonality in defects

The aim of this exercise is to find out if there are any common factors between the raised defects that can be focused on, which can then lead to improvement in quality.

The general idea is to obtain comments of all flagged defects from relevant Projects through API, used Pandas to store them and then utilise NLP to see if classification can be done.

As Atlassian JIRA store comments under each issue, and there are multiples issues being flagged, there needs to be repeated calls to the API. Considering the size, and time taken for each API call, time can be cut down for the exercise by utilising multi-threading.

## Set Up

In [ ]:
!pip3 install atlassian-python-api

In [ ]:
!pip install XlsxWriter # For writing to Excel

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
from atlassian import JIRA
import requests
import json

# For running threading to make operations run faster
import concurrent
from concurrent.futures import ThreadPoolExecutor

In [ ]:
from bs4 import BeautifulSoup
!pip install lxml
import lxml

# import reges
import re

# NLTK package
import string
from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, regexp_tokenize

In [ ]:
# Set credentials
cred_auth = {
    "Username": "someuser@email.com", # JIRA Account ID
    "Token": "SomeAPIToken", # JIRA Personal Access Token
    "Encoded_Pass": "SomeBase64EncodedToken" # JIRA username:token Base64 encoding
}

# Declare jira instance
jira_instance = JIRA(
    url = "https://jirainstance.com/",
    token = cred_auth['Token']
)

# Set query string
searchApi_string = "https://jirainstance.net/rest/api/2/search?jql="

## Methods

In [ ]:
def writeDFToExcel(df, filename):
  '''
  Method to write to excel based on given dataframe
  Also use xlsx writer to set the column width so that the document doesn't open at fixed width
  And hide some columns that can be useful
  '''
  with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Report', encoding='utf-8-sig', index=False)
    worksheet = writer.sheets['Report']

In [ ]:
def get_all_mainquery_results_api(auth, fieldlist, query_string: str, user=None)->list:
  '''
  Method to retrieve all requests from a specific JIRA JQL request
  Not restricted by the page limit count
  By taking in the credentials, any fields filter, the query string
  and outputting in a dataframe the result of the JSON response
  '''
  issues_per_query = 100
  list_of_jira_issues = []

  # Get the total issues in the results set. This is one extra request
  num_issues_in_query_result_set = jira_instance.jql(query_string, limit=0)['total']
  print(f"Query {query_string} returns {num_issues_in_query_result_set} issues.")

  queryString_utf = query_string.replace(" ", "+").replace("'", "%22") # Replace all spaces and inverted commans in URL string

  # Use floor division + 1 to calculate the number of requests needed
  for query_number in range(0, (num_issues_in_query_result_set // issues_per_query) + 1):
    startNum = query_number * issues_per_query
    results = requests.get(
        url = "https://jirainstance.com/rest/api/2/search?jql=" + queryString_utf,
        headers = {
            "Authorization": "Bearer " + cred_auth["Token"],
            "Content-Type": "application/json"
        },
        params = {'fields': fieldlist, 'maxResults': issues_per_query, 'startAt': startNum}
    )

    project_issueList = json.loads(results.text)
    list_of_jira_issues.extend(project_issueList["issues"])
    print(startNum, issues_per_query * (query_number+1)) # Print out which iteration of the loop this is in

  df = pd.json_normalize(list_of_jira_issues) # Json serialise into dataframe

  # Define fields of interest
  fields_of_interest = ["fields.project.name", "fields.issuetype.name"]

  # Filter only to display fields of interest
  df_filtered = df.loc[:, df.columns.intersection(fields_of_interest)]

  if user != None:
    df_filtered['Actioned By'] = user # Create column to specify who was the one who actioned

  return df_filtered

In [ ]:
def get_comment_results_api(auth, issue_id)-> list:
  '''
  Method to get all the comments within a JIRA issue
  Takes in the credentials and the issue ID of a JIRA e.g. AKE-1212
  Returns a dataframe of all comments
  '''
  issues_per_query = 100
  list_of_comments = []

  # Get the total issues in the results set. This is one extra request
  num_issues_in_query_result_set = jira_instance.issue_get_comments(issue_id)["total"]

  # Use floor division + 1 to calculate the number of requests needed
  for query_number in range(0, (num_issues_in_query_result_set // issues_per_query) + 1):
    startNum = query_number * issues_per_query
    results = requests.get(
        url = "https://jirainstance.com/rest/api/2/issue/" + issue_id + "/comment",
        headers = {
            "Authorization": "Bearer " + cred_auth["Token"],
            "Content-Type": "application/json"
        },
        params = {'startAt': startNum}
    )

    comment_list = json.loads(results.text)
    list_of_comments.extend(comment_list["comments"])

  comment_df = pd.json_normalize(list_of_comments) # Json serialise into dataframe

  # declare new sequence to store cleaned text of comments
  cleanedText_df = []

  # Use BeautifulSoup to parse the html text and remove tags / strings
  for idx, commentRow in enumerate(comment_df['body']):
    soup = BeautifulSoup(commentRow, 'lxml')

    # Strip of tags and data decompose
    for data in soup(['style', 'script', 'code']):
      data.decompose() # remove tags

    textList = list(soup.stripped_strings)

    # Remove midstring trailing spaces and unicode spaces
    for i, content in enumerate(textList):
      textList[i] = content.replace('\r\n', ' ').replace('\xa0', ' ')

    cleanedText_df.append(' '.join(textList))

  # Concatenate the dataframes together
  final_df = pd.concat([comment_df[['author.displayName', 'created', 'updated']], pd.DataFrame({'commentCleaned': cleanedText_df})], axis=1)
  final_df['updated'] = final_df['updated'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f%z'))
  final_df['updated'] = pd.to_datetime(final_df['updated'], utc=True)
  final_df['updatedDateOnly'] = final_df['updated'].dt.date
  final_df['MainIssue'] = issue_id

  return final_df


## Set up Date Range

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd
set_days_ago = 7

today_date = date.today()

# Calculate 7 days ago from today
seven_days_start = today_date + relativedelta(days= -set_days_ago)
timeStart_date = date(seven_days_start.year, seven_days_start.month, seven_days_start.day)
timeEnd_date = date(today_date.year, today_date.month, today_date.day)
print("Starting Date " + str(timeStart_date) + "\nEnding Date: " + str(timeEnd_date))

# Claculate last week start and end
start_date = today_date + timedelta(-today_date.weekday(), weeks=-1)
end_date = today_date + timedelta(-today_date.weekday() - 1)

print("Last Week Start " + str(start_date) + "\nLast Week End: " + str(end_date))

## Retrieve Issues DataFrame by list of users

In [ ]:
eeList = ['EmployeeName', 'EmployeeName2', 'EmployeeName3']

queryString = "issue in updatedBy({}, '" + start_date.strftime("%Y/%m/%d") + "', '" + end_date.strftime("%Y/%m/%d") + "')"

In [ ]:
# Get list of issues updated by the members, if any, within the stipulated time

with ThreadPoolExecutor(max_workers=30) as executor:
  # With each row in the Dataframe, we set as a future in the thread
  futures = [
      executor.submit(get_all_mainquery_results_api, cred_auth, field_final, queryString.format("'" + user + "'"), user) for user in eeList
  ]
  concurrent.futures.wait(futures)

  df_MainTasksList = pd.DataFrame({'key': [], 'Summary': [], 'Deployed Date': [], 'Description': [], 'Assignee': [], 'country_json': []})

  for f in concurrent.futures.as_completed(futures):
    try:
      data = f.result()
    except Exception as exc:
      print('%r generated an exception: %s' % (data, exc))
    else:
      if (not(data.empty)):
        df_MainTasksList = pd.concat([df_MainTasksList, data])

## Data cleaning

In [ ]:
# Drop duplicates
df_finalMainSet = df_MainTasksList.drop_duplicates(subset=['key']).reset_index(drop=True)

In [ ]:
# Replace label column into concatenated string
countryCol = []

for index, row in df_finalMainSet.iterrows():
  country = []

  if row['country_json'] is None:
    countryCol.append(None)
  else:
    for countryEntry in row['country_json']:
      country.append(countryEntry['value'])

    countryString = ','.join(country)
    countryCol.append(countryString)

In [ ]:
df_finalMainSet.insert(6, 'country', countryCol)
df_finalMainSet = df_finalMainSet.drop(columns=['country_json'])

## Retrieve Comments Dataframe from list of Issues

In [ ]:
# Get list of comments for every issues
with ThreadPoolExecutor(max_workers=30) as execturo:
  # with each row in the DataFrame, set as future in thread
  futures = [
      executor.submit(get_comment_results_api, cred_auth, row['key'] for idx,row in df_finalMainSet.iterrows())
  ]
  concurrent.futures.wait(futures)

  df_completeCommentList = pd.DataFrame({'MainIssue': [], 'author.displayName': [], 'created': [], 'updated': [], 'commentCleaned': [], 'UpdatedDateOnly': []})

  for f in concurrent.futures.as_completed(futures):
    try:
      data = f.result()
    except Exception as exc:
      print('%r generated an exception: %s' % (data,exc))
    else:
      if not(data.empty):
        df_completeCommentList = pd.concat([df_completeCommentList, data])

## Data Cleaning round 2

In [ ]:
# Merge with original issues to identify comments to issue
df_workOn = pd.merge(df_completeCommentList, df_finalMainSet[['key', 'Description']], how='inner', left_on='MainIssue', right_on='key').drop_duplicates(subset=['key']).reset_index(drop=True)

# Remove references to PR
df_workOn = df_workOn[df_workOn['commentCleaned'].str.contains('tfs')]

# Remove all other duplicates of same issue since only interested in the first instance
df_workOn.sort_values(by=['UpdatedDateOnly'], inplace=True)
df_workOn.drop_duplicates(subset=['MainIssue', 'author.displayName'], inplace=True)

## NLTK Preprocessing

In [ ]:
# Start with Preprocessing

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

# Initialise the text preprocessing tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [2]:
def process_TFStext(text):
  '''
  Method taking in the comments and tokenizing through regex
  This identifies if the string with overview exists which represents
  the comment is for a PR
  '''
  newToken = ''
  tokens = regexp_tokenize(text, pattern=r"\s|.,;'", gaps=True) # Split out words by the given delimiters
  alphaOrPunc = [word for word in tokens is word.isalpha() or word in string.punctuation or word.isnumeric()]
  remain = [word for word in tokens if word not in alphaOrPunc] # Only keep tokenize words that are not alpha or puncutation or numeric
  remain = [identify_tfs(word) for word in remain] # Keep
  remain = [word for word in remain if word] # remove empty strings from list

  # When stripping html tags, whitespaces were removed leading to mash-up of words, so try to split out again
  # Check that overview is the last possible word
  for index, word in enumerate(remain):
    if word.find('overview') > 0 and word.find('overview') + 8 < len(word):
      remain[index] = word[:word.find('overview') + 8]

  final = ",".join(remain)

  if final:
    final = final
  else:
    final = None

  return final

def identify_tfs(word):
  '''
  Method that takes in a word and search if there exists "tfs"
  Return the actual URL text instead of the markdown with square brackets
  '''
  index = word.find('https://')
  html = ''
  if index > 0:
    word = word[index:]

  if re.search('\w\/tfs\w?', word):
    html = word.replace('[', '').replace(']', '')
  else:
    html = ''

  return html

In [ ]:
# Create new column to store all TFS url
df_workOn['TFS_Url'] = df_workOn['commentCleaned'].apply(process_TFStext)

In [ ]:
def removeTFS(urlString, TextString):
  '''
  Method that takes in both url and the comment string
  and returns the comment string without the url
  '''
  newString = TextString
  for url in urlString.split(','):
    newString = newString.replace(url, '')

  return newString

def removeAllSquareBracketText(TextString):
  '''
  Method that removes all square brackets
  and return comment string without it
  '''
  newString = TextString
  matches = re.findall(r'\[(.*?)\]', TextString) # Obtain list of text in square brackets
  for url in matches:
    newString = newString.replace(url, '')

  return newString

def removeAllJIRATicketReferences(TextString):
  '''
  Method that takes in comment string
  and returns comment string without JIRA ticket
  '''
  newString = TextString
  matches = re.findall(r'[\w]{4,}-[0-9]{1,}', TextString) # Obtain list of possible JIRA tickets
  for url in matches:
    newString = newString.replace(url, '')

  return newString

In [ ]:
df_workOn['RemainingText'] = df_workOn['commentCleaned']

# Remove any links / tags caused by square brackets
df_workOn['RemainingText'] = df_workOn['RemainingText'].apply(removeAllSquareBracketText)

# Remove TFS URL. Because it is currently in an array, split by delimiter and run across each element against the string
df_workOn['RemainingText'] = df_workOn.apply(lambda x: removeTFS(x['TFS URL'], x['RemainingText']), axis=1)

# Remove all Jira tickets
df_workOn['RemainingText'] = df_workOn['RemainingText'].apply(removeAllJIRATicketReferences)

In [ ]:
# Text Preprocessing

def get_wordnet_pos(treebank_tag):
  '''
  Method that maps POS tag to first character used by WordNetLemmatizer
  '''
  if treebank_tag.startswith('J'):
    return wordnet.ADJ
  elif treebank_tag.startswith('V'):
    return wordnet.VERB
  elif treebank_tag.startswith('N'):
    return wordnet.NOUN
  elif treebank_tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

def remove_html(word):
  '''
  Method that takes in a word and goes through all checks
  replacing without the checks
  '''
  # print("tfs", word, re.search('\A\/tfs\w?', word))
  # print("_aoverview", word, re.search('\A_a=\w?', word))
  # print("special char", word, re.search('^[^a-zA-Z0-9]*$', word))
  # print("^ at start", word, re.search('^[\^]\w+', word))
  # print("check for file cases", word, re.search('\w*\.[a-zA-Z]{3, 4}', word))
  # print("check for jira tagging via ~", word, re.search('\w?~\w+', word))

  if re.search('\A\/tfs\w?', word):
    return ''
  elif re.search('\A_a=\w?', word):
    return ''
  elif re.search('^[^a-zA-Z0-9]*$', word):
    return ''
  elif re.search('^[\^]\w+', word):
    return ''
  elif re.search('\w*\.[a-zA-Z]{3, 4}', word):
    return ''
  elif re.search('\w?~\w+', word):
    return word

In [ ]:
def preprocess_description(text):
  '''
  Method taking in the comment string
  and preprocessing into simple words
  '''
  tokens = word_tokenize(text)
  tokens=[word.lower() for word in tokens]
  tokens = [word for word in tokens if word.isalpha() or word in string.punctuation]
  pos_tags = nltk.pos_tag(tokens)
  tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word,pos in pos_tags]
  tokens = [word for word in tokens if word not in stop_words]

  return " ".join(tokens)

df_workOn['DescriptionCleaned'] = df_workOn['Description'].apply(preprocess_description)

In [ ]:
# Find common themes in those that currently fail the test
def most_common(df, top_n=10):
  list_to_exclude = ['please', 'would', 'team', 'thanks', 'thank', 'hope', 'client']
  name_to_exclude = ['tom', 'harry']
  all_words = [word for word in ' '.join(df['DescriptionCleaned']).split() if word not in string.puncutation and word not in list_to_exclude and word not in name_to_exclude]
  word_counts = Counter(all_words)
  top_10_words = word_counts.most_common(top_n)

  return top_10_words

list_top10 = most_common(df_workOn)

def findWithinTop10(TextString):
  # for each top10 words, compare against the list and see if it's within
  reasonsList = []
  for word in list_top10:
    if re.search(word[0], TextString):
      reasonsList.append(word[0])

  return ",".join(reasonsList)

In [ ]:
df_workOn['PossibleReason'] = df_workOn['DescriptionCleaned'].apply(findWithinTop10)

## Export

In [ ]:
# Final clean-up
df_workOn = df_workOn.drop(columns=['index', 'RemainingText', 'DescriptionCleaned'])
df_workOn = df_workOn.rename(columns={'author.displayName': 'Author', 'commentCleaned': 'OriginalComment', 'UpdatedDateOnly': 'Date'})

In [ ]:
timestamp_str = start_date.strftime('%Y%m%d')

filename = timestamp_str + '_CommentsCleanedJIRA' + '.xlsx'

writeDFToExcel(df_workOn, filename)

# Summary

Through the NLTK package, I can tokenize the comments and find the 10 most common bag of words and then compare it against individual comment to classify the reasons.

Problems I ran into includes how raw the JIRA data is and how much cleaning I had to do before the comments are considered ready. Furthermore, as this involves comments, there are also instances of names and markdown that needs to be carefully removed. A lot of Regex was utilised on the comment strings to slowly strip the excess information, which took a lot of time.

Overall though, the project was able to highlight about 40% of the issues correctly after reviewing which is not super high, but enough to find out there is some patterns in the defects being raised.